In [3]:
import os
from shutil import copyfile
import glob

# Deep leearnig and machine learning libraries (Keras and scikit learn)
from os import listdir
from tensorflow.keras import datasets, layers, models, utils
from keras.utils import to_categorical
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix , classification_report
# from sklearn.model_selection import cross_val_score
from sklearn.model_selection import  KFold
# Classic libraries with python
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Settings:
img_size = 64
num_classes = 10
path ='/content/drive/MyDrive/Dataset'



def read_greyimg(data_file):
    #Normalization step1 (convert image to grey)
    img = cv2.imread(data_file,cv2.IMREAD_GRAYSCALE )
    img = cv2.resize(img, (img_size, img_size))
    return img

def first_normalize():
  X = []
  Y = [] 
  labels = listdir(path) 
  for i, label in enumerate(labels):
          dir = path+'/'+label
          for data_file in listdir(dir):
              img = read_greyimg(dir+'/'+data_file)
              X.append(img)
              Y.append(label)
  X = np.array(X)
  Y = np.array(Y)
  Y = to_categorical(Y, num_classes)

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8)
  # Normalization step2 (Divide each image by 255)
  X_train=X_train/255
  X_test=X_test/255

  #convert 3D to 2D 
  X_train  = X_train.reshape( -1, img_size*img_size)
  X_test= X_test.reshape( -1, img_size*img_size)
  return X_train,Y_train,X_test,Y_test

In [11]:
 # first Neural Network architecture
 X_train ,Y_train,X_test,Y_test = first_normalize() 

 ann1 = models.Sequential([
           layers.Dense(2048,input_shape=(img_size*img_size,), activation='relu'),
           layers.Dense(1024, activation='relu'),
           layers.Dense(10, activation='softmax'),
 ])

 ann1.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])


# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i=0
for train_index, test_index in kfolds.split(X_cross):
    i=i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    ann1.fit(X_train_cross, Y_train_cross,epochs=8)
    acc=ann1.evaluate(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(acc[1]))



#Evaluate model 1
ann1.summary()
Y_pred1 = ann1.predict(X_test)
Y_pred1 = [np.argmax(element) for element in Y_pred1]
Y_test1 = [np.argmax(element) for element in Y_test]
print(classification_report(Y_test1, Y_pred1)) 

Epoch 1/8
38/38 [==============================] - 3s 70ms/step - loss: 4.5766 - accuracy: 0.1125
Epoch 2/8
38/38 [==============================] - 3s 71ms/step - loss: 2.2728 - accuracy: 0.1675
Epoch 3/8
38/38 [==============================] - 3s 70ms/step - loss: 2.1175 - accuracy: 0.2250
Epoch 4/8
38/38 [==============================] - 3s 71ms/step - loss: 1.8059 - accuracy: 0.3408
Epoch 5/8
38/38 [==============================] - 3s 70ms/step - loss: 1.6119 - accuracy: 0.3992
Epoch 6/8
38/38 [==============================] - 3s 69ms/step - loss: 1.4536 - accuracy: 0.4800
Epoch 7/8
38/38 [==============================] - 3s 69ms/step - loss: 1.3235 - accuracy: 0.5358
Epoch 8/8
19/19 [==============================] - 1s 21ms/step - loss: 1.3184 - accuracy: 0.5067
acc in split 1 :0.5066666603088379
Epoch 1/8
38/38 [==============================] - 3s 69ms/step - loss: 1.2872 - accuracy: 0.5300
Epoch 2/8
38/38 [==============================] - 3s 69ms/step - loss: 1.1750 - ac

In [ ]:
# Second Neural Network architecture
X_train ,Y_train,X_test,Y_test = first_normalize() 

ann2 = models.Sequential([
  layers.Dense(100,input_shape=(img_size*img_size,), activation='relu'),
  layers.Dense(400,input_shape=(img_size*img_size,), activation='sigmoid'),
  layers.Dense(300,input_shape=(img_size*img_size,), activation='sigmoid'),
  layers.Dense(10, activation='softmax'),
])

ann2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i=0
for train_index, test_index in kfolds.split(X_cross):
    i=i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    ann2.fit(X_train_cross, Y_train_cross,epochs=8)
    acc=ann2.evaluate(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(acc[1]))


#Evaluate model 2
ann2.summary()

Y_pred2 = ann2.predict(X_test)
Y_pred2 = [np.argmax(element) for element in Y_pred2]
Y_test2 = [np.argmax(element) for element in Y_test]
print(classification_report(Y_test2, Y_pred2))  


Epoch 1/8
38/38 [==============================] - 1s 6ms/step - loss: 2.3736 - accuracy: 0.0958
Epoch 2/8
38/38 [==============================] - 0s 6ms/step - loss: 2.3489 - accuracy: 0.1075
Epoch 3/8
38/38 [==============================] - 0s 6ms/step - loss: 2.2813 - accuracy: 0.1225
Epoch 4/8
38/38 [==============================] - 0s 6ms/step - loss: 2.1323 - accuracy: 0.2150
Epoch 5/8
38/38 [==============================] - 0s 6ms/step - loss: 1.8750 - accuracy: 0.2908
Epoch 6/8
38/38 [==============================] - 0s 6ms/step - loss: 1.6593 - accuracy: 0.3958
Epoch 7/8
38/38 [==============================] - 0s 7ms/step - loss: 1.5379 - accuracy: 0.4117
Epoch 8/8
19/19 [==============================] - 0s 3ms/step - loss: 1.5917 - accuracy: 0.4117
acc in split 1 :0.4116666615009308
Epoch 1/8
38/38 [==============================] - 0s 7ms/step - loss: 1.3607 - accuracy: 0.5083
Epoch 2/8
38/38 [==============================] - 0s 7ms/step - loss: 1.2913 - accuracy: 0.

In [2]:
#Train using normalized RGB image

def read_rgbimg(data_file):
    img = cv2.imread(data_file)
    img = cv2.resize(img, (img_size, img_size))
    return img
def second_normalize():
  X = []
  Y = []
  labels = listdir(path) 
  for i, label in enumerate(labels):
          dir = path+'/'+label
          for data_file in listdir(dir):
              img = read_rgbimg(dir+'/'+data_file)
              X.append(img)
              Y.append(label)
  X = np.array(X).astype('float32')
  Y = np.array(Y)
  Y = to_categorical(Y, num_classes) 

  average= X.mean(axis=0)
  X -= average
  X=X/255
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8)
  return X_train,Y_train,X_test,Y_test


In [5]:
#CNN model
X_train ,Y_train,X_test,Y_test = second_normalize() 

cnn1 = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(4, 4), activation='relu', input_shape=(img_size,img_size,3)),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
cnn1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i = 0
for train_index, test_index in kfolds.split(X_cross):
    i = i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    cnn1.fit(X_train_cross, Y_train_cross,epochs=5)
    acc=cnn1.evaluate(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(acc[1]))




#Evaluate cnn
cnn1.summary()

Y_pred = cnn1.predict(X_test)
Y_pred = [np.argmax(element) for element in Y_pred]
Y_test = [np.argmax(element) for element in Y_test]
print(classification_report(Y_test, Y_pred))  

Epoch 1/5
38/38 [==============================] - 6s 153ms/step - loss: 1.3379 - accuracy: 0.5700
Epoch 2/5
38/38 [==============================] - 6s 153ms/step - loss: 0.5042 - accuracy: 0.8350
Epoch 3/5
38/38 [==============================] - 6s 151ms/step - loss: 0.2375 - accuracy: 0.9358
Epoch 4/5
38/38 [==============================] - 6s 152ms/step - loss: 0.1085 - accuracy: 0.9725
Epoch 5/5
19/19 [==============================] - 1s 41ms/step - loss: 0.5707 - accuracy: 0.8517
acc in split 1 :0.8516666889190674
Epoch 1/5
38/38 [==============================] - 6s 154ms/step - loss: 0.3329 - accuracy: 0.9058
Epoch 2/5
38/38 [==============================] - 6s 153ms/step - loss: 0.0884 - accuracy: 0.9775
Epoch 3/5
38/38 [==============================] - 6s 150ms/step - loss: 0.0313 - accuracy: 0.9917
Epoch 4/5
38/38 [==============================] - 6s 150ms/step - loss: 0.0117 - accuracy: 0.9992
Epoch 5/5
19/19 [==============================] - 1s 38ms/step - loss: 0.0

In [ ]:
#SVM vs NN 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

X_train ,Y_train,X_test,Y_test = second_normalize() 
Y_train=np.argmax(Y_train,axis=1)
Y_test=np.argmax(Y_test,axis=1)
X_train  = X_train.reshape( -1, img_size*img_size*3)#in case of rgb image 
X_test= X_test.reshape( -1, img_size*img_size*3)  

svm = SVC() 
# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i = 0
for train_index, test_index in kfolds.split(X_cross):
    i = i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    svm.fit(X_train_cross,Y_train_cross)
    score = svm.score(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(score))


Y_predict= svm.predict(X_test)
print(classification_report(Y_test, Y_predict))
 


acc in split 1 :0.8166666666666667
acc in split 2 :0.8416666666666667
acc in split 3 :0.845
              precision    recall  f1-score   support

           0       0.90      0.93      0.91        46
           1       0.89      0.91      0.90        45
           2       0.76      0.83      0.79        41
           3       0.97      0.86      0.91        43
           4       0.82      0.67      0.74        49
           5       0.96      0.96      0.96        54
           6       0.82      0.84      0.83        43
           7       0.72      0.81      0.76        47
           8       0.82      0.84      0.83        43
           9       0.87      0.85      0.86        40

    accuracy                           0.85       451
   macro avg       0.85      0.85      0.85       451
weighted avg       0.85      0.85      0.85       451

